# Kafka Data Consumer

## I/ Objectifs du notebook

## II/ Mécanisme de collecte en streaming des données


In [1]:
import time
import sys
sys.path.append("../TimeSeriesTools")

In [2]:
kafka_server ='localhost:9092'
#kafka_server ='islin-hdpnod1:6667'
#kafka_server ='islin-hdplnod06:6667'

In [3]:
import kairosdb_utils
global kairosdb_server 
kairosdb_server = "http://localhost:9080"

In [4]:
import influxdb_utils
db_host= 'localhost'
port = '8086'
db_name='TimeSeriesBench'
influxdb_client = influxdb_utils.influxdb_connect(db_host, port)

Trying to connect to InfluxDB server without proxy: localhost on port: 8086
connection sucess!


In [5]:
import warp10_utils
global warp10_server 
warp10_server = "http://localhost:8080"

In [6]:
from kafka import KafkaConsumer, TopicPartition
def consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,insert_data,convert_to_dbdata,validate_data):
    print("CONNECTION KAFKA : ",kafka_server)
    print("TOPIC            : ",topic)
    print("DB NAME          : ",db_name)
    print("COLLECTION NAME  : ",coll_name)
    
    scheme = get_scheme(topic)
    consumer = KafkaConsumer(bootstrap_servers=kafka_server)

    tp = TopicPartition(topic, 0)
    consumer.assign([tp])
    consumer.seek_to_end(tp)
    lastOffset = consumer.position(tp)
    print("RANGE:",tp,lastOffset)
    consumer.seek_to_beginning(tp)
    # end
    try :
        while True:
            list_data = []
            count=0
            for message in consumer:
                msg = message.value.decode()
                count += 1
                list_data.append(msg)
                if len(list_data) == bucket_size:
                    #print(count,str(len(list_data)) + " data fetched from topic : " + topic)
                    new_data = convert_to_dbdata(list_data,scheme,validate_data)
                    insert_data(db_name, coll_name, new_data)
                    list_data.clear()
                if count == lastOffset:
                    #print(count,str(len(list_data)) + " data fetched from topic : " + topic)
                    new_data = convert_to_dbdata(list_data,scheme,validate_data)
                    insert_data(db_name, coll_name, new_data)
                    list_data.clear()
                    return

    finally:
        # Close down consumer to commit final offsets.
        consumer.close()

## III/ Mécanismes de validation et corrections des données

In [7]:
def get_scheme(topic):
    if "SmartGrid" in topic:
        scheme = {"timestamp":{'type': 'string','required': True,'empty': False},
                  "tagname":{'type': 'string','required': True,'empty': False},
                  "value":{'type': 'string','required': True,'empty': False},
                  "quality":{'type': 'string','required': True,'empty': False}}
        return scheme
    if "WindProp" in topic :
        scheme = {  "Heure":{'type': 'string','required': True,'empty': False},
                    "Temps écoulé":{'type': 'string','required': True,'empty': False},
                    '4069 state':{'type': 'string','required': True,'empty': False},
                    'Battery voltage':{'type': 'string','required': True,'empty': False},
                    'Flow SP':{'type': 'string','required': True,'empty': False},
                    'Unit Code':{'type': 'string','required': True,'empty': False},
                    'Flow M':{'type': 'string','required': True,'empty': False},
                    'Pressure1':{'type': 'string','required': True,'empty': False},
                    'Temperature1':{'type': 'string','required': True,'empty': False},
                    "Latitude":{'type': 'string','required': True,'empty': False},
                    "Longitude":{'type': 'string','required': True,'empty': False},
                    "Altitude":{'type': 'string','required': True,'empty': False},
                    "Head_Rel_True North":{'type': 'string','required': True,'empty': False},
                    "Pressure2":{'type': 'string','required': True,'empty': False},
                    "Temperature2":{'type': 'string','required': True,'empty': False},
                    "Humidity":{'type': 'string','required': True,'empty': False},
                    "MDA Wnd Dir":{'type': 'string','required': True,'empty': False},
                    "MDA Wnd Speed":{'type': 'string','required': True,'empty': False},
                    "MWD Wind Dir":{'type': 'string','required': True,'empty': False},
                    "MWD Wind Speed":{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 1':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 2':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 3':{'type': 'string','required': True,'empty': False},
                    'Gaz Concentration 4':{'type': 'string','required': True,'empty': False},
                    'SPA 1':{'type': 'string','required': True,'empty': False},
                    'SPA 2':{'type': 'string','required': True,'empty': False},
                    'Cellule Photo':{'type': 'string','required': True,'empty': False},
                    'Temperature3':{'type': 'string','required': True,'empty': False},
                    'Pressure3':{'type': 'string','required': True,'empty': False},
                    'Flow MassFlow 1':{'type': 'string','required': True,'empty': False},
                    'NOTUSED Flow MassFlow 2':{'type': 'string','required': True,'empty': False},
                    'Flow':{'type': 'string','required': True,'empty': False},
                    'Humidity2':{'type': 'string','required': True,'empty': False},
                    'Test':{'type': 'string','required': True,'empty': False},
                    'Details':{'type': 'string','required': True,'empty': False},
                    'SPA 3':{'type': 'string','required': True,'empty': False},
                    'SPA 4':{'type': 'string','required': True,'empty': False},
                    "CavityPressure":{'type': 'string','required': True,'empty': False},
                    "CavityTemp":{'type': 'string','required': True,'empty': False},
                    "CH4":{'type': 'string','required': True,'empty': False},
                    "CH4_dry":{'type': 'string','required': True,'empty': False},
                    "C2H6":{'type': 'string','required': True,'empty': False},
                    "C2H6_dry":{'type': 'string','required': True,'empty': False},
                    "13CH4":{'type': 'string','required': True,'empty': False},
                    "H2O":{'type': 'string','required': True,'empty': False},
                    "CO2":{'type': 'string','required': True,'empty': False},
                    "C2C1Ratio":{'type': 'string','required': True,'empty': False},
                    "Delta_iCH4_Raw":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_30s":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_2min":{'type': 'string','required': True,'empty': False},
                    "HP_Delta_iCH4_5min":{'type': 'string','required': True,'empty': False} }
        return scheme

In [8]:
def clean_data(data,scheme):
    from cerberus import Validator
    v = Validator(scheme)
    for index,item in enumerate(data,start=0):
        res = v.validate(item)
        if (res == False):
            print("corrupt data in line :",index,", error : ",v.errors)
            del data[index]

## IV/ Mécanismes de mise en base des données

In [9]:
import mongodb_utils
db_host= 'localhost'
port = '27018'
db_name='TimeSeriesBench'
mongodb_client = mongodb_utils.mongodb_connect(db_host, port)

Trying to connect to MongoDB server: localhost on port: 27018


### A/ Mise en base du schéma des données

In [10]:
def str_to_unix(date):
    from datetime import datetime
    dt = datetime.strptime(date, '%d/%m/%Y %H:%M:%S')
    epoch = datetime.utcfromtimestamp(0)
    return int((dt - epoch).total_seconds()) * 1000

In [11]:
def type_convert(data):
    data = data.replace(',', '.')
    time = str_to_unix(data.split(";", 1)[0].split(":", 1)[1])
    tags = data.split(";", 2)[1]
    data_treated = [tags,[time, data.split(";", 2)[2]]]
    return data_treated

In [12]:
def mongodb_convert_to_dbdata(list_data,scheme,validate_data):
    col= [k for k in scheme.keys()]
    new_dict=[]
    
    t0 = time.process_time()
    for item in list_data:
        info=item.split(":",1)[1].split(";")
        dict_item ={col[i]:info[i] for i in range(len(col))}
        new_dict.append(dict_item)
    if validate_data:
        clean_data(new_dict,scheme)
    t1 = time.process_time()
    #print('.. %f seconds for validate' % (t1 - t0))
    
    return new_dict

In [13]:
def db_convert_to_dbdata(list_msg,scheme,validate_data):
    from cerberus import Validator
    col = [k for k in scheme.keys()]
    if validate_data:
        v = Validator(scheme)
    db_data = []
    t0 = time.process_time()
    for index,item in enumerate(list_msg):
        if validate_data:
            info = item.split(":", 1)[1].split(";")
            dict_item = {col[i]: info[i] for i in range(len(col))}
            res = v.validate(dict_item)
        else:
            res = True
        if res == True:
            new_data = type_convert(item)
            db_data.append(new_data)
        else:
            print("corrupt data in line :",index,", error : ",v.errors)
    t1 = time.process_time()
    #print('.. %f seconds for validate' % (t1 - t0))
    return db_data

In [14]:
def mongodb_insert_data(db_name, coll_name, list_data):
    mongodb_utils.insert_many_docs(mongodb_client,db_name, coll_name,list_data)    
    return len(list_data)

In [15]:
def kairosdb_insert_data(db_name, coll_name, list_data):
    tags_data_list = {}
    for d in list_data:
        if d[0] in tags_data_list.keys():
            tags_data_list[d[0]].append(d[1])
        else:
            tags_data_list[d[0]] = [d[1]]
    
    tag_keys = ['Buiding','Device','Measure']
    for i,(k,v) in enumerate(tags_data_list.items()):
        tag_values = k.split(".")
        tags = {tag_keys[i]:tag_values[i] for i in range(len(tag_keys))}
        kairosdb_utils.insert_many_docs_with_tags(kairosdb_server,db_name, coll_name,tags,v)    
    return len(list_data)

In [16]:
def influxdb_insert_data(db_name, coll_name, list_data):
    tags_data_list = {}
    for d in list_data:
        if d[0] in tags_data_list.keys():
            tags_data_list[d[0]].append(d[1])
        else:
            tags_data_list[d[0]] = [d[1]]
    
    tag_keys = ['Buiding','Device','Measure']
    for i,(k,v) in enumerate(tags_data_list.items()):
        tag_values = k.split(".")
        tags = {tag_keys[i]:tag_values[i] for i in range(len(tag_keys))}
        influxdb_utils.insert_many_docs_with_tags(influxdb_client,db_name, coll_name,tags,v)    
    return len(list_data)

In [17]:
def warp10_insert_data(db_name, coll_name, list_data):
    tags_data_list = {}
    for d in list_data:
        if d[0] in tags_data_list.keys():
            tags_data_list[d[0]].append(d[1])
        else:
            tags_data_list[d[0]] = [d[1]]
    
    tag_keys = ['Buiding','Device','Measure']
    for i,(k,v) in enumerate(tags_data_list.items()):
        tag_values = k.split(".")
        tags = {tag_keys[i]:tag_values[i] for i in range(len(tag_keys))}
        warp10_utils.insert_many_docs_with_tags(warp10_server,db_name, coll_name,tags,v)    
    return len(list_data)

#### 1) Cas test SmartGrid

In [18]:
topic = 'SmartGridDataCryolite20190101OneMonthTopic'
scheme = get_scheme(topic)

In [19]:
coll_name='SmartGridCryolite20190101OneMonthV10000'

In [20]:
db=mongodb_client[db_name]
db.list_collection_names()

['WindPropLacqOneDayP25-B1000',
 'WindPropLacqOneDayP25',
 'SmartGridCryolite20190101OneMonthV10000',
 'schemes',
 'SmartGridCryolite20190101OneMonth',
 'SmartGridCryolite20190101OneMonthV2',
 'WindPropLacqOneDay',
 'WindPropLacqOneDayV2',
 'SmartGridCryolite20190101OneMonthV1']

In [ ]:
scheme_collection = db['schemes']

In [ ]:
scheme

In [ ]:
scheme_doc = { "name" : "SmartGrid" , "value" : scheme }

In [ ]:
scheme_id = scheme_collection.insert_one(scheme)

In [ ]:
scheme_id

In [21]:
%%time
bucket_size = 10000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,mongodb_insert_data,mongodb_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  SmartGridDataCryolite20190101OneMonthTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  SmartGridCryolite20190101OneMonthV10000
RANGE: TopicPartition(topic='SmartGridDataCryolite20190101OneMonthTopic', partition=0) 769598
CPU times: user 24 s, sys: 479 ms, total: 24.4 s
Wall time: 28.1 s


In [22]:
%%time
bucket_size = 10000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,kairosdb_insert_data,db_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  SmartGridDataCryolite20190101OneMonthTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  SmartGridCryolite20190101OneMonthV10000
RANGE: TopicPartition(topic='SmartGridDataCryolite20190101OneMonthTopic', partition=0) 769598
CPU times: user 27.6 s, sys: 316 ms, total: 27.9 s
Wall time: 1min 38s


In [23]:
%%time
bucket_size = 10000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,influxdb_insert_data,db_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  SmartGridDataCryolite20190101OneMonthTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  SmartGridCryolite20190101OneMonthV10000
RANGE: TopicPartition(topic='SmartGridDataCryolite20190101OneMonthTopic', partition=0) 769598
CPU times: user 39.4 s, sys: 436 ms, total: 39.8 s
Wall time: 47.5 s


In [24]:
%%time
bucket_size = 10000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,warp10_insert_data,db_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  SmartGridDataCryolite20190101OneMonthTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  SmartGridCryolite20190101OneMonthV10000
RANGE: TopicPartition(topic='SmartGridDataCryolite20190101OneMonthTopic', partition=0) 769598
CPU times: user 32.2 s, sys: 417 ms, total: 32.6 s
Wall time: 42.8 s


### 2) Cas test WindProp

In [25]:
def type_convert(data):
    data = data.replace(',', '.')
    time = str_to_unix(data.split(";", 1)[0].split(":", 1)[1])
    data_treated = [time, data.split(";", 1)[1]]
    return data_treated

In [26]:
def db_convert_to_dbdata(list_msg,scheme,validate_data):
    from cerberus import Validator
    col = [k for k in scheme.keys()]
    if validate_data:
        v = Validator(scheme)
    db_data = []
    t0 = time.process_time()
    for index,item in enumerate(list_msg):
        if validate_data:
            info = item.split(":", 1)[1].split(";")
            dict_item = {col[i]: info[i] for i in range(len(col))}
            res = v.validate(dict_item)
        else:
            res = True
        if res == True:
            new_data = type_convert(item)
            db_data.append(new_data)
        else:
            print("corrupt data in line :",index,", error : ",v.errors)
    t1 = time.process_time()
    #print('.. %f seconds for validate' % (t1 - t0))
    return db_data

In [27]:
def kairosdb_insert_data(db_name, coll_name, list_data):
    kairosdb_utils.insert_many_docs(kairosdb_server,db_name, coll_name,list_data)    
    return len(list_data)

In [28]:
def influxdb_insert_data(db_name, coll_name, list_data):
    influxdb_utils.insert_many_docs(influxdb_client,db_name, coll_name,list_data)    
    return len(list_data)

In [29]:
def warp10_insert_data(db_name, coll_name, list_data):
    warp10_utils.insert_many_docs(warp10_server,db_name, coll_name,list_data)    
    return len(list_data)

In [30]:
topic = 'WindPropDataLacqOneDayP25NewTopic'

In [31]:
scheme = get_scheme(topic)

In [32]:
coll_name='WindPropLacqOneDayP25-B1000'

In [ ]:
db = mongodb_client[db_name]
scheme_collection = db['schemes']

In [ ]:
scheme_id = scheme_collection.insert_one(scheme)

In [ ]:
scheme_id

In [33]:
%%time
bucket_size = 1000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,mongodb_insert_data,mongodb_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  WindPropDataLacqOneDayP25NewTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  WindPropLacqOneDayP25-B1000
RANGE: TopicPartition(topic='WindPropDataLacqOneDayP25NewTopic', partition=0) 7288
CPU times: user 455 ms, sys: 6.11 ms, total: 461 ms
Wall time: 710 ms


In [34]:
%%time
bucket_size = 1000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,kairosdb_insert_data,db_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  WindPropDataLacqOneDayP25NewTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  WindPropLacqOneDayP25-B1000
RANGE: TopicPartition(topic='WindPropDataLacqOneDayP25NewTopic', partition=0) 7288
CPU times: user 306 ms, sys: 11 ms, total: 317 ms
Wall time: 1.41 s


In [35]:
%%time
bucket_size = 1000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,influxdb_insert_data,db_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  WindPropDataLacqOneDayP25NewTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  WindPropLacqOneDayP25-B1000
RANGE: TopicPartition(topic='WindPropDataLacqOneDayP25NewTopic', partition=0) 7288
CPU times: user 426 ms, sys: 12.1 ms, total: 438 ms
Wall time: 1.01 s


In [36]:
%%time
bucket_size = 1000
validate_data = False
consumer_single_collection(kafka_server,topic,db_name, coll_name,bucket_size,warp10_insert_data,db_convert_to_dbdata,validate_data)

CONNECTION KAFKA :  localhost:9092
TOPIC            :  WindPropDataLacqOneDayP25NewTopic
DB NAME          :  TimeSeriesBench
COLLECTION NAME  :  WindPropLacqOneDayP25-B1000
RANGE: TopicPartition(topic='WindPropDataLacqOneDayP25NewTopic', partition=0) 7288
CPU times: user 409 ms, sys: 6 ms, total: 415 ms
Wall time: 1.08 s


### 3) Cas test Pollution